In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler , LabelEncoder
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.over_sampling import RandomOverSampler 

In [4]:
# Proses melakukan import dataset yang diperlukan
test_set = pd.read_csv('dataset/test_feature.csv')
features = pd.read_csv('dataset/train_features.csv')
labels = pd.read_csv('dataset/train_label.csv')

In [6]:
features.head()

,facilities,rating,location
0,RestaurantBARSwimmingPools,7.8 Very GoodFrom 10 reviews,Stokol
1,intrnetRestaurantgym,5.6 GoodFrom 4 reviews,Machlessvile
2,restaurantgympoolBar,7.2 Very GoodFrom 38 reviews,Wanderland
3,BARRestaurant,7.3 Very GoodFrom 6 reviews,Uberlandia
4,InternetRestaurant,7.2 Very GoodFrom 30 reviews,Stokol


In [5]:
# Menampilkan jumlah missing value pada features
na_feature = features.isna().sum()
na_feature

facilities    301
rating        637
location        0
dtype: int64

In [7]:
# Proses mengisi missing values string pada features dengan 'NULL'
features = features.fillna('NULL')

In [ ]:
features['Restaurant']=''